# 🧠 Experimentos Deep Learning - Sinal Raw

## Objetivo
Avaliar modelos de Deep Learning usando o sinal raw (sem pré-processamento wavelet):
- **CNN** (Convolutional Neural Network)
- **LSTM** (Long Short-Term Memory)
- **CNN-LSTM** (Híbrido)
- **Transformer**

## Pipeline
1. Carregar dados
2. Preparar para DL (adicionar dimensão de canal)
3. Treinar cada modelo com early stopping
4. Avaliar no conjunto de teste
5. Comparar resultados

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

# TensorFlow
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponível: {tf.config.list_physical_devices('GPU')}")

# Imports locais
import sys
sys.path.append('.')
from src.models import (
    create_cnn_model, create_lstm_model, 
    create_cnn_lstm_model, create_transformer_model,
    get_callbacks
)
from src.evaluation import RegressionEvaluator, ResultsManager
from src.visualization import ExperimentVisualizer
from config.experiment_config import (
    DATA_DIR, RESULTS_DIR, MODELS_DIR,
    DL_TRAINING_CONFIG, DL_MODELS_CONFIG
)

# Configuração
plt.style.use('seaborn-v0_8-whitegrid')
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
(RESULTS_DIR / "dl_raw_experiments").mkdir(exist_ok=True)

print("\n✅ Imports realizados com sucesso!")

## 1. Carregar e Preparar Dados

In [ ]:
# Carregar datasets
X_train = np.load(DATA_DIR / "X_train.npy")
y_train = np.load(DATA_DIR / "y_train.npy")
X_val = np.load(DATA_DIR / "X_val.npy")
y_val = np.load(DATA_DIR / "y_val.npy")
X_test = np.load(DATA_DIR / "X_test.npy")
y_test = np.load(DATA_DIR / "y_test.npy")

# Adicionar dimensão de canal para CNN/LSTM
X_train = X_train[..., np.newaxis]  # (N, seq_len, 1)
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

print(f"📦 Dados Carregados (com canal):")
print(f"  Train: X={X_train.shape}, y={y_train.shape}")
print(f"  Val:   X={X_val.shape}, y={y_val.shape}")
print(f"  Test:  X={X_test.shape}, y={y_test.shape}")

input_shape = X_train.shape[1:]
print(f"\nInput shape para modelos: {input_shape}")

## 2. Configuração

In [ ]:
# Gerenciadores
results_manager = ResultsManager(RESULTS_DIR / "dl_raw_experiments")
evaluator = RegressionEvaluator()
visualizer = ExperimentVisualizer()

# Configuração de treinamento
training_config = DL_TRAINING_CONFIG.copy()
print("Configuração de Treinamento:")
for k, v in training_config.items():
    print(f"  {k}: {v}")

# Armazenar resultados
all_results = {}
all_histories = {}

## 3. Experimento 1: CNN

In [ ]:
print("="*70)
print("🔵 Experimento: CNN com Sinal Raw")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
cnn_params = DL_MODELS_CONFIG['CNN']
model_cnn = create_cnn_model(input_shape, params=cnn_params)
model_cnn.summary()

# Callbacks
model_path = str(MODELS_DIR / "raw_cnn_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_cnn = model_cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_cnn = model_cnn.predict(X_test, verbose=0).flatten()

# Métricas
cnn_metrics = evaluator.evaluate(y_test, y_pred_cnn)

print(f"\n📊 Resultados CNN (Raw):")
print(f"  RMSE: {cnn_metrics['rmse']:.6f}")
print(f"  MAE:  {cnn_metrics['mae']:.6f}")
print(f"  R²:   {cnn_metrics['r2']:.6f}")
print(f"  Tempo: {elapsed:.2f}s, Epochs: {len(history_cnn.history['loss'])}")

all_results['Raw_CNN'] = {
    'metrics': cnn_metrics,
    'time': elapsed,
    'epochs': len(history_cnn.history['loss']),
    'y_pred': y_pred_cnn,
    'params': model_cnn.count_params()
}
all_histories['Raw_CNN'] = history_cnn.history

results_manager.log_experiment(
    'DL_Raw', 'CNN', cnn_metrics,
    {'params': cnn_params}
)

## 4. Experimento 2: LSTM

In [ ]:
print("="*70)
print("🔵 Experimento: LSTM com Sinal Raw")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
lstm_params = DL_MODELS_CONFIG['LSTM']
model_lstm = create_lstm_model(input_shape, params=lstm_params)
model_lstm.summary()

# Callbacks
model_path = str(MODELS_DIR / "raw_lstm_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_lstm = model_lstm.predict(X_test, verbose=0).flatten()

# Métricas
lstm_metrics = evaluator.evaluate(y_test, y_pred_lstm)

print(f"\n📊 Resultados LSTM (Raw):")
print(f"  RMSE: {lstm_metrics['rmse']:.6f}")
print(f"  MAE:  {lstm_metrics['mae']:.6f}")
print(f"  R²:   {lstm_metrics['r2']:.6f}")
print(f"  Tempo: {elapsed:.2f}s, Epochs: {len(history_lstm.history['loss'])}")

all_results['Raw_LSTM'] = {
    'metrics': lstm_metrics,
    'time': elapsed,
    'epochs': len(history_lstm.history['loss']),
    'y_pred': y_pred_lstm,
    'params': model_lstm.count_params()
}
all_histories['Raw_LSTM'] = history_lstm.history

results_manager.log_experiment(
    'DL_Raw', 'LSTM', lstm_metrics,
    {'params': lstm_params}
)

## 5. Experimento 3: CNN-LSTM

In [ ]:
print("="*70)
print("🔵 Experimento: CNN-LSTM com Sinal Raw")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
cnn_lstm_params = DL_MODELS_CONFIG['CNN_LSTM']
model_cnn_lstm = create_cnn_lstm_model(input_shape, params=cnn_lstm_params)
model_cnn_lstm.summary()

# Callbacks
model_path = str(MODELS_DIR / "raw_cnn_lstm_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_cnn_lstm = model_cnn_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_cnn_lstm = model_cnn_lstm.predict(X_test, verbose=0).flatten()

# Métricas
cnn_lstm_metrics = evaluator.evaluate(y_test, y_pred_cnn_lstm)

print(f"\n📊 Resultados CNN-LSTM (Raw):")
print(f"  RMSE: {cnn_lstm_metrics['rmse']:.6f}")
print(f"  MAE:  {cnn_lstm_metrics['mae']:.6f}")
print(f"  R²:   {cnn_lstm_metrics['r2']:.6f}")
print(f"  Tempo: {elapsed:.2f}s, Epochs: {len(history_cnn_lstm.history['loss'])}")

all_results['Raw_CNN_LSTM'] = {
    'metrics': cnn_lstm_metrics,
    'time': elapsed,
    'epochs': len(history_cnn_lstm.history['loss']),
    'y_pred': y_pred_cnn_lstm,
    'params': model_cnn_lstm.count_params()
}
all_histories['Raw_CNN_LSTM'] = history_cnn_lstm.history

results_manager.log_experiment(
    'DL_Raw', 'CNN_LSTM', cnn_lstm_metrics,
    {'params': cnn_lstm_params}
)

## 6. Experimento 4: Transformer

In [ ]:
print("="*70)
print("🔵 Experimento: Transformer com Sinal Raw")
print("="*70)

tf.keras.backend.clear_session()

# Criar modelo
transformer_params = DL_MODELS_CONFIG['Transformer']
model_transformer = create_transformer_model(input_shape, params=transformer_params)
model_transformer.summary()

# Callbacks
model_path = str(MODELS_DIR / "raw_transformer_best.keras")
callbacks = get_callbacks(model_path, patience_early=15, patience_lr=7)

# Treinar
t0 = time.time()
history_transformer = model_transformer.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=training_config['epochs'],
    batch_size=training_config['batch_size'],
    callbacks=callbacks,
    verbose=1
)
elapsed = time.time() - t0

# Predições
y_pred_transformer = model_transformer.predict(X_test, verbose=0).flatten()

# Métricas
transformer_metrics = evaluator.evaluate(y_test, y_pred_transformer)

print(f"\n📊 Resultados Transformer (Raw):")
print(f"  RMSE: {transformer_metrics['rmse']:.6f}")
print(f"  MAE:  {transformer_metrics['mae']:.6f}")
print(f"  R²:   {transformer_metrics['r2']:.6f}")
print(f"  Tempo: {elapsed:.2f}s, Epochs: {len(history_transformer.history['loss'])}")

all_results['Raw_Transformer'] = {
    'metrics': transformer_metrics,
    'time': elapsed,
    'epochs': len(history_transformer.history['loss']),
    'y_pred': y_pred_transformer,
    'params': model_transformer.count_params()
}
all_histories['Raw_Transformer'] = history_transformer.history

results_manager.log_experiment(
    'DL_Raw', 'Transformer', transformer_metrics,
    {'params': transformer_params}
)

## 7. Comparação dos Resultados

In [ ]:
# Criar DataFrame comparativo
comparison_data = []
for model_name, result in all_results.items():
    row = {
        'Model': model_name,
        'RMSE': result['metrics']['rmse'],
        'MAE': result['metrics']['mae'],
        'R²': result['metrics']['r2'],
        'Params': result['params'],
        'Time (s)': result['time'],
        'Epochs': result['epochs']
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('RMSE')

print("\n" + "="*70)
print("📊 COMPARAÇÃO FINAL - Deep Learning com Sinal Raw")
print("="*70)
print(comparison_df.to_string(index=False))

# Salvar
comparison_df.to_csv(RESULTS_DIR / "dl_raw_experiments" / "comparison_dl_raw.csv", index=False)

In [ ]:
# Visualização comparativa
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

metrics_to_plot = ['RMSE', 'MAE', 'R²']
colors = plt.cm.tab10.colors

for idx, metric in enumerate(metrics_to_plot):
    data = comparison_df.set_index('Model')[metric].sort_values(
        ascending=(metric != 'R²')
    )
    bars = axes[idx].barh(data.index, data.values, color=colors[:len(data)])
    axes[idx].set_xlabel(metric)
    axes[idx].set_title(f'Comparação: {metric}')
    axes[idx].grid(True, alpha=0.3, axis='x')
    
    for bar, val in zip(bars, data.values):
        axes[idx].text(val, bar.get_y() + bar.get_height()/2,
                      f'{val:.4f}', va='center', ha='left', fontsize=9)

plt.suptitle('Deep Learning com Sinal Raw', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / "dl_raw_experiments" / "comparison_dl_raw.png", dpi=150, bbox_inches='tight')
plt.show()

## 8. Histórico de Treinamento

In [ ]:
# Plot de históricos de treinamento
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, (model_name, history) in enumerate(all_histories.items()):
    ax = axes[idx]
    ax.plot(history['loss'], label='Train Loss')
    ax.plot(history['val_loss'], label='Val Loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss (MSE)')
    ax.set_title(f'{model_name} - Training History')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Histórico de Treinamento - DL Raw', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / "dl_raw_experiments" / "training_history.png", dpi=150, bbox_inches='tight')
plt.show()

## 9. Análise de Predições

In [ ]:
# Encontrar melhor modelo
best_model_name = comparison_df.iloc[0]['Model']
best_result = all_results[best_model_name]

print(f"\n🏆 Melhor Modelo: {best_model_name}")

# Plot de predições
fig = visualizer.plot_prediction_comparison(
    y_test, best_result['y_pred'],
    model_name=best_model_name,
    n_samples=500,
    save_path=RESULTS_DIR / "dl_raw_experiments" / f"predictions_{best_model_name}.png"
)
plt.show()

## 10. Resumo

In [ ]:
print("\n" + "="*70)
print("📋 RESUMO - Experimentos DL com Sinal Raw")
print("="*70)
print(f"\n✅ Modelos avaliados: {len(all_results)}")
print(f"✅ Melhor modelo: {best_model_name}")
print(f"✅ Melhor RMSE: {comparison_df.iloc[0]['RMSE']:.6f}")
print(f"✅ Melhor R²: {comparison_df.iloc[0]['R²']:.6f}")
print(f"\n📁 Resultados salvos em: {RESULTS_DIR / 'dl_raw_experiments'}")
print("\n🎉 Notebook concluído com sucesso!")